In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Assuming data is in a DataFrame called df and the columns are named as per the sample provided
# Load data
df = pd.read_csv('/content/tweet dataset.csv')  # Correct the file path

# Use the correct column name for reviews and labels
X = df['text']  # Text column
y = df['target']  # Label column, assuming '0' for negative and '1' for positive sentiment

# Remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

# Apply stopword removal
X = X.apply(remove_stopwords)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenization and sequence padding
vocab_size = 1000
oov_tok = "<OOV>"
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

sequence_length = 120
train_padded = pad_sequences(train_sequences, maxlen=sequence_length, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=sequence_length, padding='post', truncating='post')

# Model definition
model = Sequential([
    Embedding(vocab_size, 16, input_length=sequence_length),
    LSTM(32),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit model with callbacks to avoid overfitting
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3),
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)
]
history = model.fit(train_padded, y_train, epochs=50, validation_data=(test_padded, y_test), callbacks=callbacks)

# Output model summary and history
print(model.summary())
metrics_df = pd.DataFrame(history.history)
print(metrics_df)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Epoch 1/50
3/3 [==============================] - 3s 351ms/step - loss: 0.6511 - accuracy: 0.0000e+00 - val_loss: 0.6469 - val_accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - ETA: 0s - loss: 0.5910 - accuracy: 0.0000e+00

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3/3 [==============================] - 0s 76ms/step - loss: 0.5910 - accuracy: 0.0000e+00 - val_loss: 0.6095 - val_accuracy: 0.0000e+00
Epoch 3/50
3/3 [==============================] - 0s 75ms/step - loss: 0.5141 - accuracy: 0.0000e+00 - val_loss: 0.5606 - val_accuracy: 0.0000e+00
Epoch 4/50
3/3 [==============================] - 0s 79ms/step - loss: 0.4244 - accuracy: 0.0000e+00 - val_loss: 0.4912 - val_accuracy: 0.0000e+00
Epoch 5/50
3/3 [==============================] - 0s 78ms/step - loss: 0.2859 - accuracy: 0.0000e+00 - val_loss: 0.3827 - val_accuracy: 0.0000e+00
Epoch 6/50
3/3 [==============================] - 0s 75ms/step - loss: 0.0305 - accuracy: 0.0000e+00 - val_loss: 0.1876 - val_accuracy: 0.0000e+00
Epoch 7/50
3/3 [==============================] - 0s 72ms/step - loss: -0.4517 - accuracy: 0.0000e+00 - val_loss: -0.1500 - val_accuracy: 0.0000e+00
Epoch 8/50
3/3 [==============================] - 0s 77ms/step - loss: -1.1610 - accuracy: 0.0000e+00 - val_loss: -0.4692 - val

In [10]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_padded, y_test)

# Print the results
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


1/1 [==============================] - 0s 35ms/step - loss: -6.5186 - accuracy: 0.0000e+00
Test Loss: -6.518629550933838
Test Accuracy: 0.0
